# Scripts

In this notebook we present all possible scripts.

In [1]:
import math
import polars as pl
from mammos_mmag.materials import Materials
from mammos_mmag.parameters import Parameters
from mammos_mmag.simulation import Simulation

pl.Config.set_fmt_str_lengths(200)  # Display str value
pl.Config.set_tbl_rows(-1)  # Display all rows

polars.config.Config

## Mesh, materials, simulation parameters

### Mesh

We load the mesh of a cube enclosed with a sphere. This sphere is embedded in a further spherical shell.

This mesh was generated by Salome in the `unv` mesh, and successively converted in the `fly` format. As an input to the `Simulation` class we give the attribute `mesh_filepath`.

### Material parameters

We define the material parameters, defined by 3 domains: the internal geometry (a cube), the sphere and the infinity shell.

### Simulation parameters

We finally define all the simulation parameters we will use.
> Note: most of these parameters are the "default" parameters and in fact we do not need to define them.

In [2]:
sim = Simulation(
    mesh_filepath="data/cube.fly",
    materials=Materials(
        domains=[
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 4e-7 * math.pi * 4.9e06,
                "K2": 0.0,
                "Js": 1.61,
                "A": 4e-7 * math.pi * 8.0e-11,
            },
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 0.0,
                "K2": 0.0,
                "Js": 0.0,
                "A": 0.0,
            },
            {
                "theta": 0.0,
                "phi": 0.0,
                "K1": 0.0,
                "K2": 0.0,
                "Js": 0.0,
                "A": 0.0,
            },
        ],
    ),
    parameters=Parameters(
        size=1.0e-9,
        scale=0,
        m_vect=[0, 0, 1],
        hstart=1,
        hfinal=-1,
        hstep=-0.2,
        h_vect=[0.01745, 0, 0.99984],
        mstep=0.4,
        mfinal=-1.2,
        tol_fun=1e-10,
        tol_hmag_factor=1,
        precond_iter=10,
    ),
)
sim

Simulation(mesh_filepath=PosixPath('data/cube.fly'), materials=Materials(domains=[MaterialDomain(theta=0.0, phi=0.0, K1=6.157521601035994, K2=0.0, Js=1.61, A=1.0053096491487338e-16), MaterialDomain(theta=0.0, phi=0.0, K1=0.0, K2=0.0, Js=0.0, A=0.0), MaterialDomain(theta=0.0, phi=0.0, K1=0.0, K2=0.0, Js=0.0, A=0.0)]), parameters=Parameters(size=1e-09, scale=0.0, state='mxyz', m_vect=[0.0, 0.0, 1.0], hmag_on=1, hstart=1.0, hfinal=-1.0, hstep=-0.2, h_vect=[0.01745, 0.0, 0.99984], mstep=0.4, mfinal=-1.2, iter_max=1000, precond_iter=10, tol_fun=1e-10, tol_hmag_factor=1.0, tol_u=1e-10, verbose=0))

Note that all of this could also have been defined using file paths:
```python
sim = Simulation(
    mesh_filepath="data/cube.fly",
    materials_filepath="data/cube.krn",
    parameters="data/cube.p2",
)
```

## Available scripts
For all scripts we can specify the optional variables `outdir` and `name`. While the first identifies the output directory where the input and output files will be store (and where the script is executed), the `name` variable defines the output file names.

### Save the mesh and the materials

To create the `vtk` file for the visualisation of the material properties we can use the script `materials`.

In [3]:
sim.run_materials(outdir="out/materials", name="cube")

This create discretized representations of the material scalar functions and fields in the mesh.

In [4]:
sim.materials_fields

<meshio mesh object>
  Number of points: 9698
  Number of cells:
    tetra: 53275
  Cell data: A, Js, K, tags, u
  Field data: TIME, CYCLE

This object contains information about the mesh and the scalar and vectorial fields defined on it.
The mesh points are accessible with

In [5]:
sim.materials_fields.points

array([[ 10.      ,  10.      , -10.      ],
       [ 10.      ,  10.      ,  10.      ],
       [ 10.      , -10.      , -10.      ],
       ...,
       [  1.504399,  -9.061382,  25.99511 ],
       [-27.87612 ,  -6.595955,   6.178888],
       [ -4.684083,  14.47755 ,  23.06286 ]])

While the field is, e.g.,

In [6]:
sim.materials_fields.cell_data["A"]

[array([[0.00000e+00],
        [1.00531e-16],
        [1.00531e-16],
        ...,
        [0.00000e+00],
        [0.00000e+00],
        [0.00000e+00]])]

Representing the value at each mesh point.

### Compute the magnetostatic field

To create the `vtk` file for visualisation of the magnetic scalar potential and the magnetic field we use the script `hmag`.
With linear basis function for the magnetic scalar potential $u$, the magnetostatic field $h = - \nabla u$ is defined at the finite elements. By smoothing the field can be transfered to the nodes of the finite element mesh. This is `h_at_nodes`.

In [7]:
sim.run_hmag(outdir="out/hmag", name="cube")

The output will be stored in `sim.hmag` as a `meshio` object.

In [8]:
sim.hmag

<meshio mesh object>
  Number of points: 9698
  Number of cells:
    tetra: 53275
  Point data: U, h_nodes, m
  Cell data: h, tags
  Field data: TIME, CYCLE

And in particular, to retrieve the demagnetization field we can use

In [9]:
sim.hmag.point_data["h_nodes"]

array([[-0.31572  , -0.366436 , -0.3361191],
       [ 0.3719234,  0.3562195, -0.2583703],
       [-0.3796821,  0.3288842, -0.2831827],
       ...,
       [-0.       , -0.       , -0.       ],
       [-0.       , -0.       , -0.       ],
       [-0.       , -0.       , -0.       ]])

The scripts creates two file: the magnetostatic field, as seen above, will be stored in `cube_hmag.vtu`.
At the same time the software also gives the magnetostatic energy density computed with finite elements and compares it with the analytic solution:
- `from field`:
  \begin{equation}
      E_{\mathsf{field}} := \frac{1}{2} \int_\Omega \frac{\mathbf{h} \cdot J_s \mathbf{m}}{V} \ \mathrm{d}x,
  \end{equation}
  where $\Omega$ is the domain, $\mathbf{h}$ is the demagnetization field, $J_s$ is the spontaneeous polarization, $\mathbf{m}$ is the magnetization field, and $V$ is the volume of the domain.
- `from_gradient`:
  \begin{equation}
      \frac{1}{2} \sum_i \mathbf{m}_i \cdot \mathbf{g}_i,
  \end{equation}
  where $\mathbf{m}_i$ and $\mathbf{g}_i$ are the unit vector of the magnetization and the gradient of the energy normalized by the volume of the energy with respect to $\mathbf{m}_i$ at the nodes of the finite element mesh.
- `analytic`: $J_s^2 / (6 \mu_0)$

This information will be saved in `cube.csv`:

In [10]:
pl.read_csv("out/hmag/cube.csv", skip_rows=1)

name,value,explanation
str,f64,str
"""E_field""",323030.603623,"""Energy density evaluated from field (J/m^3)."""
"""E_gradient""",323030.603623,"""Energy density evaluated from gradient (J/m^3)."""
"""E_analytic""",343787.940036,"""Energy density evaluated analytically (J/m^3)."""


### Exchange and anisotropy energy

To test the computation of the exchange and anisotropy energy density we can use the script `exani`.

This gives the exchange energy density of a vortex in the $xy$-plane and the anistropy energy density in the uniformly magnetized state.
Here we have placed the anistropy direction paralle to to the $z$-axis. The anisotropy energy density is calculated as $-K (\mathbf{m} \cdot \mathbf{k})^2$  where $\mathbf{m}$ is the unit vector of magnetization and $\mathbf{k}$ is the anisotropy direction. $K$ is the magnetocrystalline anisotropy constant

In [11]:
sim.run_exani(outdir="out/exani", name="cube")

The accuracy of this script is then analyzed for two different magnetization, a vortex and a uniform vector.

In [12]:
pl.read_csv("out/exani/cube_vortex.csv", skip_rows=1)

name,value,explanation
str,f64,str
"""E_gradient""",983624.055448,"""Energy evaluated from gradient (J/m^3)."""
"""E_analytic""",986960.440109,"""Energy evaluated analytically (J/m^3)."""


In [13]:
pl.read_csv("out/exani/cube_uniform.csv", skip_rows=1)

name,value,explanation
str,f64,str
"""E_gradient""",-4.9000e6,"""Energy evaluated from gradient (J/m^3)."""
"""E_analytic""",-4.9000e6,"""Energy evaluated analytically (J/m^3)."""


### Zeeman energy

The script `external` calculates the Zeeman energy density for an external field of $\mu_0 H_{\mathsf{ext}} = 1.2 \ T$ by finite elements and analytically.

In [14]:
sim.run_external(outdir="out/external", name="cube")

The generated energy densities are found in the generated file `cube.csv`:

In [15]:
pl.read_csv("out/external/cube.csv", skip_rows=1)

name,value,explanation
str,f64,str
"""E_gradient""",-1.5372e6,"""Energy evaluated from gradient (J/m^3)."""
"""E_analytic""",-1.5372e6,"""Energy evaluated analytically (J/m^3)."""


### jax implementation

The above tools checked the energy calculation with the finite element backend.
From the finite element backend system matrices are generated for micromagnetic simulations.
The script `mapping` is used to test the energy calculations with matrices.

The `mmag` software uses sparse matrix methods from `jax`.

In [16]:
sim.run_mapping(outdir="out/mapping", name="cube")

Information about the calculation are stored in different files. Among them, `cube_energy.csv` stores the total energy density for the uniformly magnetized state:

In [17]:
pl.read_csv("out/mapping/cube_energy.csv", skip_rows=1)

name,value,explanation
str,f64,str
"""E_jax""",-6.1142e6,"""Energy evaluated with jax backend (J/m^3)."""
"""E_analytic""",-6.1142e6,"""Energy evaluated analytically (J/m^3)."""


The file `cube_stats.txt`, on the other hand, shows information about memory and runtime.

In [18]:
with open("out/mapping/cube_stats.txt") as file:
    print(file.read())

MAP FINITE ELEMENT BACKEND (esys-escript) TO JAX.
Memory before escript2jax: 181.55078125 MB.
Memory after  escript2jax: 471.8203125 MB.
Memory after garbage collection: 471.8203125 MB.
Timing and statistics.
elapsed time: 0.23759937286376953
function_calls: 1


### Storing sparse matrices

The sparse matrices used for computation can be stored and reused for simulations with the same finite element mesh. To store the matrices use the script `store`.

In [19]:
sim.run_store(outdir="out/store", name="cube")

### Demagnetization curve - Hysteresis Loop

To compute the demagnetization curve we use the script `loop`.

In [20]:
sim.run_loop(outdir="out/loop", name="cube")

--> demag 1.0 1.6096814901982122
--> demag 0.8 1.6096688701022313
--> demag 0.6000000000000001 1.609655432558706
--> demag 0.4000000000000001 1.609641101269194
--> demag 0.20000000000000007 1.6096257900457398
--> demag 5.551115123125783e-17 1.6096094032620407
--> demag -0.19999999999999996 1.6095918323404776
--> demag -0.39999999999999997 1.6095729541997776
--> demag -0.6 1.609552628787997
--> demag -0.8 1.6095306961070202
--> demag -1.0 1.6095069726689042

--> done
elaplsed time             63.51559805870056
iterations                32
  function calls          75


This creates the file `cube.dat` which gives the demagnetization curve. The columns of the file are:
- `vtk number`: the number of the `vtk` file that corresponds to the field and magnetic polarisation values in the line.
- `mu0 Hext`: the value of $\mu_0 H_{\mathsf{ext}}$ ($T$) where $\mu_0$ is the permability of vacuum and $H_{\mathsf{ext}}$ is the external value of the external field.
- `polarisation`: the componenent of magnetic polarisation ($T$) parallel to the direction of the external field.
- `energy density`: the energy density ($J/m^3$) of the current state.

In [21]:
pl.read_csv("out/loop/cube.dat", separator=" ", has_header=False)

column_1,column_2,column_3,column_4
i64,f64,f64,f64
0,1.0,1.609681,-5.8595e6
1,0.8,1.609669,-5.6034e6
1,0.6,1.609655,-5.3472e6
1,0.4,1.609641,-5.0910e6
1,0.2,1.609626,-4.8348e6
1,5.5511e-17,1.609609,-4.5786e6
1,-0.2,1.609592,-4.3224e6
1,-0.4,1.609573,-4.0663e6
1,-0.6,1.609553,-3.8101e6


Further execution statistics are found in file `cube_stats.txt`

In [22]:
with open("out/loop/cube_stats.txt") as file:
    print(file.read())

Memory before escript2jax: 181.5625 MB.
Memory after  escript2jax: 449.96484375 MB.
Memory after garbage collection: 449.96484375 MB.


And `vtu` files for different iterations are found in `sim.loop_vtu_list`.
For example,

In [23]:
sim.loop_vtu_list[0]

<meshio mesh object>
  Number of points: 9698
  Number of cells:
    tetra: 53275
  Point data: m, u
  Cell data: tags
  Field data: TIME, CYCLE